In [6]:
import torch
a =torch.ones(2,5,1)
a

tensor([[[1.],
         [1.],
         [1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.],
         [1.],
         [1.]]])

In [10]:
a.sum(dim=1).long().squeeze().shape

torch.Size([2])

In [6]:
import torch

# Example input
x = torch.rand(1,3,6)

# Duplicate along the time (T) dimension
x_duplicated = x.repeat_interleave(1, dim=1)  # Repeat each timestep 2 times

print(x_duplicated)
print(x_duplicated.shape)  # (1, 6, 1)


tensor([[[0.0816, 0.3557, 0.3862, 0.5614, 0.7836, 0.3444],
         [0.8075, 0.5739, 0.2482, 0.2364, 0.2070, 0.5204],
         [0.5498, 0.2703, 0.3211, 0.2806, 0.4379, 0.0161]]])
torch.Size([1, 3, 6])


In [7]:
x

tensor([[[0.0816, 0.3557, 0.3862, 0.5614, 0.7836, 0.3444],
         [0.8075, 0.5739, 0.2482, 0.2364, 0.2070, 0.5204],
         [0.5498, 0.2703, 0.3211, 0.2806, 0.4379, 0.0161]]])

In [6]:
with open("/raid/home/rajivratn/hemant_rajivratn/last/data/txt/train.wrd", "r") as f:
        sentences = f.readlines()
sentences = [s.strip() for s in sentences if len(s.strip()) > 0]

import numpy as np
# word to character ratio 
word_to_char_ratio = np.mean([len(sentence.split()) / len(sentence) for sentence in sentences])
print("Word to character ratio:", word_to_char_ratio)

# character to word ratio
char_to_word_ratio = np.mean([len(sentence) / len(sentence.split()) for sentence in sentences])
print("Character to word ratio:", char_to_word_ratio)


Word to character ratio: 0.18955613922824477
Character to word ratio: 5.3258137107501025


In [8]:
import re
import numpy as np
# Path to your log file
log_file_paths = [ "/raid/home/rajivratn/hemant_rajivratn/grpo/src/logs_0/training_2025-06-03_16-14-43.log", "/raid/home/rajivratn/hemant_rajivratn/grpo/src/logs_0.001/training_2025-06-03_16-09-52.log", "/raid/home/rajivratn/hemant_rajivratn/grpo/src/logs_0.01/training_2025-06-03_16-09-34.log", "/raid/home/rajivratn/hemant_rajivratn/grpo/src/logs_0.1/training_2025-06-03_15-49-43.log", "/raid/home/rajivratn/hemant_rajivratn/grpo/src/logs_1.0/training_2025-06-03_13-51-33.log"]

for log_file_path in log_file_paths:
    # Read the file
    with open(log_file_path, "r") as file:
        log_data = file.read()

    # Extract all lines starting with 'Predicted text: '
    predicted_texts = re.findall(r"Predicted text:\s*(.*?)\n", log_data)
    real_texts = re.findall(r"Real text:\s*(.*?)\n", log_data)

    # Display or process extracted predicted texts
    rlines = []
    plines = []
    for r,p in zip(real_texts, predicted_texts):
        rlines.append(r)
        plines.append(p)

    rword_to_char_ratio = np.mean([len(lines.split()) / len(lines) for lines in rlines])
    rchar_to_word_ratio = np.mean([len(lines) / len(lines.split()) for lines in rlines])
    print(f"REAL Word to character ratio for {log_file_path.split('/')[-2]}: {rword_to_char_ratio:.4f}, {rchar_to_word_ratio:.4f}")
    
    pword_to_char_ratio = np.mean([len(lines.split()) / len(lines) for lines in plines])
    pchar_to_word_ratio = np.mean([len(lines) / len(lines.split()) for lines in plines])
    print(f"PREDICTED Word to character ratio for {log_file_path.split('/')[-2]}: {pword_to_char_ratio:.4f}, {pchar_to_word_ratio:.4f}")
    
    print("")

REAL Word to character ratio for logs_0: 0.1884, 5.3519
PREDICTED Word to character ratio for logs_0: 0.2047, 5.2488

REAL Word to character ratio for logs_0.001: 0.1884, 5.3506
PREDICTED Word to character ratio for logs_0.001: 0.1744, 6.0407

REAL Word to character ratio for logs_0.01: 0.1884, 5.3518
PREDICTED Word to character ratio for logs_0.01: 0.1201, 8.6472

REAL Word to character ratio for logs_0.1: 0.1880, 5.3590
PREDICTED Word to character ratio for logs_0.1: 0.1014, 10.3681

REAL Word to character ratio for logs_1.0: 0.1887, 5.3442
PREDICTED Word to character ratio for logs_1.0: 0.1045, 9.9629



In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

from datasets import load_dataset
from trl import GRPOTrainer, GRPOConfig

from transformers import TrainerCallback
import logging

class KLLossLoggerCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None:
            # The logs dict often contains loss and other metrics
            # Assuming KL loss is named 'kl_loss' in logs (check trainer code)
            kl_loss = logs.get('kl_loss')
            if kl_loss is not None:
                logging.info(f"Step {state.global_step} KL loss: {kl_loss}")

# Dataset
dataset = load_dataset("trl-lib/tldr", split="train")

# Reward function
def reward_func(completions, **kwargs):
    return [float(len(set(c))) for c in completions]

# Use GRPOConfig instead of TrainingArguments
grpo_config = GRPOConfig(
    per_device_train_batch_size=2,
    logging_steps=1,
    output_dir="./results",
    logging_dir="./logs",      # This enables saving logs for TensorBoard
    report_to=["tensorboard"], # Enable TensorBoard logs
    num_generations=2,
    log_level="info",
    logging_first_step=True,
    log_completions=True,
    num_completions_to_print=2,
)


trainer = GRPOTrainer(
    model="Qwen/Qwen2-0.5B-Instruct",
    reward_funcs=reward_func,
    train_dataset=dataset,
    args=grpo_config,  # <-- use GRPOConfig, not TrainingArguments
)
trainer.add_callback(KLLossLoggerCallback())


trainer.train()


/raid/home/rajivratn/anaconda3/envs/grpo/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
***** Running training *****
  Num examples = 116,722
  Num Epochs = 3
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 350,166
  Number of trainable parameters = 494,032,768


tensor(0., device='cuda:0', grad_fn=<MeanBackward0>)


Step,Training Loss
1,0.000000
2,0.000000
3,0.000000
4,0.000000
5,0.000000
6,0.000000
7,0.000100
8,0.000100


tensor(0.0005, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0004, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0004, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0007, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0009, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0013, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0026, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.0027, device='cuda:0', grad_fn=<MeanBackward0>)


KeyboardInterrupt: 

In [ ]:
import soundfile as sf
import torch
import torchaudio
import matplotlib.pyplot as plt
# Load with soundfile
waveform_np, sample_rate = sf.read("/raid/home/rajivratn/hemant_rajivratn/grpo/input.wav")

def mel_spec(waveform):
    waveform = torch.from_numpy(waveform).float().unsqueeze(0)

    # Set hop length for 50Hz frame rate
    target_frame_rate = 50
    hop_length = int(sample_rate / target_frame_rate)  # 320 if sr=16k
    n_fft = 1024
    n_mels = 80

    # Mel-spectrogram transform
    mel_spec_transform = torchaudio.transforms.MelSpectrogram(
        sample_rate=sample_rate,
        n_fft=n_fft,
        hop_length=hop_length,
        n_mels=n_mels,
        power=2.0,
        center=True,
    )

    # Compute mel and log-mel
    mel_spec = mel_spec_transform(waveform)  # (channels, n_mels, time)
    log_mel_spec = torch.log(mel_spec + 1e-6)

    print(f"log-mel shape: {log_mel_spec.shape}")
    print(f"Each second corresponds to {target_frame_rate} frames")
    # Plotting the first channel (if stereo)
    plt.figure(figsize=(12, 4))
    plt.imshow(log_mel_spec[0].cpu().numpy(), aspect='auto', origin='lower',
               interpolation='none')
    plt.colorbar(format='%+2.0f dB')
    plt.xlabel('Frames (time)')
    plt.ylabel('Mel bins')
    plt.title('Log-Mel Spectrogram')
    plt.tight_layout()
    plt.show()
    return log_mel_spec # [1, 80, 296]


mel_spec(waveform_np)


In [ ]:
94480 / 320

In [ ]:
with open("/raid/home/rajivratn/hemant_rajivratn/last/data/txt/train.wrd", "r") as f:
    sentences = f.readlines()
sentences = [s for s in sentences if len(s) > 0]

In [10]:
import kenlm

# Load the model (binary format loads faster)
model = kenlm.Model('/raid/home/rajivratn/hemant_rajivratn/grpo/ngram/charlm/5.arpa')

Loading the LM will be faster if you build a binary file.
Reading /raid/home/rajivratn/hemant_rajivratn/grpo/ngram/charlm/5.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************


In [13]:

# Sentence to score
sentence = ["this is a test", "TETSEXKURVTAMAEANUSOLIOD EZ IASEHPSIYURA DI EIMSKGLO ET EI OE E UHSHI NE EB EDBSOTLNLNOR W THNOLO T H TNACW ET H TNVLE TOTOTOPT PO N AIOASAUNE ESI NAR T EDALAC AOEPDUERSNE EMOER O EO SH RESHIA IUYENIEPMSL'EP NW EDMU N PGR R T EDEDCT EH MGL LNLE ETOLEO ODIS AEI DRNK"]
sentence = [i.replace(" ", "|").upper() for i in sentence]
sentence = [" ".join(i) for i in sentence]

for s in sentence:
    # Score (log10 probability)
    score = model.score(s, bos=False, eos=False) / len(s)
    print(f"sentence: {s}, Score: {score}")

sentence: T H I S | I S | A | T E S T, Score: -0.46006283936677156
sentence: T E T S E X K U R V T A M A E A N U S O L I O D | E Z | I A S E H P S I Y U R A | D I | E I M S K G L O | E T | E I | O E | E | U H S H I | N E | E B | E D B S O T L N L N O R | W | T H N O L O | T | H | T N A C W | E T | H | T N V L E | T O T O T O P T | P O | N | A I O A S A U N E | E S I | N A R | T | E D A L A C | A O E P D U E R S N E | E M O E R | O | E O | S H | R E S H I A | I U Y E N I E P M S L ' E P | N W | E D M U | N | P G R | R | T | E D E D C T | E H | M G L | L N L E | E T O L E O | O D I S | A E I | D R N K, Score: -1.0007251530378662


In [ ]:
counts

In [ ]:
model_name="gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [ ]:
sentences = ["123", "123"]
tokenizer(sentences, return_tensors="pt")

In [ ]:
model

In [ ]:
from collections import Counter
char_counter = Counter(
            [char for sentence in [" hello world "] for char in sentence if len(char) > 0]
        )
char_counter

In [ ]:
{
            char: count / sum(char_counter.values())
            for char, count in char_counter.items()
        }

In [ ]:
import torch
import re

# Define vocab (index 0 is blank)
vocab = ['_', 'a', 'b', 'c']
blank_id = 0

# Example model output (index sequence)
output = torch.tensor([1, 1, 0, 2, 2, 0, 2, 2, 3, 3, 0])

# Step 1: Collapse consecutive repeats
# Create a mask to keep only non-duplicate consecutive elements
mask = torch.ones_like(output, dtype=torch.bool)
mask[1:] = output[1:] != output[:-1]
collapsed = output[mask]
print(collapsed)
# Step 2: Remove blanks (i.e., tokens with index 0)
final = collapsed[collapsed != blank_id]
print(final)
# Step 3: Convert indices to characters using vocab
decoded_string = ''.join(vocab[i] for i in final.tolist())

print("Decoded string:", decoded_string)  # → 'bbc'


In [ ]:
mask

In [ ]:
import numpy as np
c = np.array(['a','b','c','d'])

In [ ]:
c[filtered[0]]

In [ ]:
from transformers import ClapProcessor, ClapModel
import torch
import torchaudio
import soundfile as sf
import numpy as np

# Load model and processor
processor = ClapProcessor.from_pretrained("laion/clap-htsat-unfused")
model = ClapModel.from_pretrained("laion/clap-htsat-unfused")

# Load and preprocess audio
def load_audio(path):
    waveform, sr = torchaudio.load(path)
    if sr != 48000:
        resampler = torchaudio.transforms.Resample(sr, 48000)
        waveform = resampler(waveform)
    return waveform

audio_path = "/raid/home/rajivratn/hemant_rajivratn/librispeech/data/train/audio/train-clean-100_126791_298_53.wav"
audio = load_audio(audio_path)

# CLAP expects mono
if audio.shape[0] > 1:
    audio = torch.mean(audio, dim=0, keepdim=True)

# Pad or truncate to 10s (CLAP expects 480000 samples)
audio = torch.nn.functional.pad(audio, (0, max(0, 480000 - audio.shape[1])))[:, :480000]



In [ ]:
"SPEAK I BEG WITHOUT DREAD OF MY DISPLEASURE SAID FRANCES RETURNING THE GOOD HUMORED SMILE OF THE TROOPER WITH THE ARCHNESS NATURAL TO HER OWN SWEET FACE THE ODORS OF YOUR KITCHEN THEN CRIED LAWTON BLUNTLY FORBID MY QUITTING THE DOMAINS UNTIL I QUALIFY MYSELF TO SPEAK WITH MORE CERTAINTY CONCERNING THE FATNESS OF THE LAND".lower()

In [ ]:

# Process audio and text
texts = ["hello how are you", "this is a dog barking", "there they say is no debt and there are a girl with there they say is no debt and there are a girl with there they say is no debt and there are a girl with there they say is no debt and there are a girl with there they say is no debt and there are a girl with there they say is no debt and there are a girl with there they say is no debt and there are a girl with there they say is no debt and there are a girl with there they say is no debt and there are a girl with there they say is no debt and there are a girl with", "speak i beg without dread of my displeasure said frances returning the good humored smile of the trooper with the archness natural to her own sweet face the odors of your kitchen then cried lawton bluntly forbid my quitting the domains until i qualify myself to speak with more certainty concerning the fatness of the land"]
inputs = processor(audios=audio.squeeze().numpy(), text=texts, return_tensors="pt", padding=True)

# Forward pass
with torch.no_grad():
    outputs = model(**inputs)
    audio_embeds = outputs.audio_embeds
    text_embeds = outputs.text_embeds

In [ ]:

# Normalize embeddings
audio_embeds = torch.nn.functional.normalize(audio_embeds, dim=-1)
text_embeds = torch.nn.functional.normalize(text_embeds, dim=-1)

# Cosine similarity
similarities = torch.matmul(audio_embeds, text_embeds.T).squeeze(0)

# Print ranked results
for text, score in sorted(zip(texts, similarities), key=lambda x: -x[1]):
    print(f"{text:<30} -> Similarity: {score.item():.4f}")


In [ ]:
import kenlm
model = kenlm.Model("/raid/home/rajivratn/hemant_rajivratn/grpo/3-gram.pruned.1e-7.arpa.gz")

sentence = "this is a test".upper()
print(sentence)

# Each tuple means “this word had log10-prob logp given the previous context; ng_len is the n-gram order (e.g. 3 means a trigram was used), and oov marks out-of-vocabulary
for (logp, ng_len, oov) in model.full_scores(sentence, bos=False, eos=False):
    print(logp, ng_len, oov)

In [ ]:
def unigram(sentences):
    for s in sentences:
        log10p = [model.score(w, bos=False, eos=False) for w in sentence.split() if model.vocab_index(w) != 0]

unigram([sentence])

In [ ]:
from collections import Counter

def get_unigram_char_probs(sentences):
    # Flatten to a list of characters
    chars = [char for sentence in sentences for char in sentence.strip()]
    
    # Count frequency of each character
    char_counts = Counter(chars)
    total_chars = sum(char_counts.values())

    # Convert to probabilities
    char_probs = {char: count / total_chars for char, count in char_counts.items()}
    
    return char_probs, chars

# Example usage
sentences = [
    'THANK YOU AGAIN MISTER DEVANT HE SAID...',
    'FOR WEEKS IT NEVER CAME TO MY TURN...',
    'ONE MAN WON PAST ME INDEED DARTING...'
]

char_probs, chars = get_unigram_char_probs(sentences)

# Print sorted for readability
for char, prob in sorted(char_probs.items()):
    print(f"'{char}': {prob:.5f}")

In [ ]:
with open("/raid/home/rajivratn/hemant_rajivratn/last/data/txt/train_norm.txt", "r") as f:
    out = [i for i in f.readlines() if len(i.strip()) > 0]

In [ ]:
from tqdm import tqdm
def preprocess_char_lm(sentences):
    """Prep a list of sentences for char-level n-gram LM training."""
    preprocessed = []
    for s in tqdm(sentences):
        s = s.strip().replace(" ", "|")
        chars = list(s)
        line = "<s> " + " ".join(chars) + " </s>"
        preprocessed.append(line)
    return preprocessed

char_lm_lines = preprocess_char_lm(out)

In [ ]:
output_path = "char_lm_input.txt"

with open(output_path, "w", encoding="utf-8") as f:
    for line in char_lm_lines:
        f.write(line + "\n")

print(f"Saved to {output_path}")


In [ ]:
import torch
@torch.jit.script
def beam_search(log_probs: torch.Tensor, beam_size: int):
    """
    Performs beam search on a tensor of log probabilities.

    Args:
        log_probs (torch.Tensor): Tensor of shape (b, t, v) containing log probabilities.
        beam_size (int): Number of beams to keep at each time step.

    Returns:
        sequences (torch.Tensor): Tensor of shape (b, beam_size, t) containing the top sequences.
        scores (torch.Tensor): Tensor of shape (b, beam_size) containing the scores of the top sequences.
    """
    
    b, t, v = log_probs.size()
    
    initial_beam_size = min(beam_size, v) # At the very first step (time step 0), we can't have more beams than the vocabulary size. This line ensures that the initial number of beams considered doesn't exceed the number of possible first tokens.

    topk_scores, topk_indices = torch.topk(log_probs[:, 0, :], initial_beam_size, dim=-1) # Returns the k largest elements of the given input tensor along a given dimension
    sequences = topk_indices.unsqueeze(-1)  # (b, initial_beam_size, 1)
    scores = topk_scores  # (b, initial_beam_size)

    for step in range(1, t):
        # Expand the current sequences with all possible next tokens
        current_log_probs = log_probs[:, step, :].unsqueeze(1)  # (b, 1, v)
        expanded_scores = scores.unsqueeze(-1) + current_log_probs  # (b, beam_size, v)
        flat_scores = expanded_scores.view(b, -1)  # (b, beam_size * v)

        # Select the top-k scores and their corresponding indices
        topk_flat_scores, topk_indices = flat_scores.topk(beam_size, dim=-1)  # (b, beam_size)
        beam_indices = topk_indices // v  # Indices of sequences to expand
        token_indices = topk_indices % v  # New tokens to append

        # Gather the sequences to expand and append the new tokens
        sequences = torch.gather(sequences, 1, beam_indices.unsqueeze(-1).expand(-1, -1, sequences.size(-1)))
        sequences = torch.cat([sequences, token_indices.unsqueeze(-1)], dim=-1)  # (b, beam_size, step+1)

        # Update the scores
        scores = topk_flat_scores

    return sequences, scores.unsqueeze(-1)



In [ ]:
batch_size = 2
sequence_length = 5
vocab_size = 3
beam_size = 2

# Simulate log probabilities
log_probs = torch.randn(batch_size, sequence_length, vocab_size).log_softmax(dim=-1)
device = torch.device('cpu')
log_probs = log_probs.to(device)


In [ ]:
log_probs[1,3:,:] = -float("1000000")

In [ ]:
log_probs

In [ ]:
# Perform beam search
sequences, scores = beam_search(log_probs, beam_size)

print("Top sequences:", sequences) # bsz, beamsize,seq_len
print("Scores:", scores) # bsz, beamsize,1

In [ ]:
sequences.shape, scores.shape

In [ ]:
path_probs = torch.gather(log_probs, 2, sequences.transpose(1,2)).transpose(1,2) # bsz, beamsize, T
path_probs

In [ ]:
mean = scores.mean(dim=1, keepdim=True)
std = scores.std(dim=1, keepdim=True)

scores = (scores - mean) / std

In [ ]:
path_probs*scores

In [ ]:
sequences # bsz,beam,T

In [ ]:
# If using PyTorch
import torch

import re
import numpy as np


vocab = [' ', "'", 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '?']
vocab = np.array(vocab)


# Precompile regex to remove blanks and collapse repeats
blank_char = re.escape('?')  # adjust if blank differs
remove_blanks = re.compile(blank_char)
collapse_repeats = re.compile(r'(.)\1+')

# Convert token sequences to strings using regex merge
def decode(arr, vocab):
    raw = [collapse_repeats.sub(r'\1', remove_blanks.sub('', ''.join(vocab[row]))) for row in arr]
    return raw


In [ ]:
# Convert token sequences to strings using regex merge
def decode(arr, vocab):
    raw = [collapse_repeats.sub(r'\1', remove_blanks.sub('', ''.join(vocab[row]))) for row in arr]
    return raw

sentences = decode(sequences[0].cpu(), vocab)
sentences

In [ ]:
sequences[0], vocab

In [ ]:
arr = sequences[1,:,:]
arr

In [ ]:
import numpy as np
vocab_arr = np.array(vocab)

decoded = [''.join(vocab_arr[row]) for row in arr]
print(decoded)  # ['abc', 'cba']


In [ ]:
[collapse_repeats.sub(r'\1', remove_blanks.sub('', ''.join(vocab[row]))) for row in arr]

In [ ]:
[''.join(vocab[row]) for row in arr]

In [ ]:
decode(arr, vocab)

In [ ]:
'-'.join(vocab[np.array(arr)])

In [ ]:
import numpy as np

# Step 1: vocab and array of indices
vocab = ['a', 'b', 'c', 'd', 'e']
vocab_arr = np.array(vocab)  # shape (V,)
arr = np.array([
    [0, 1, 2],
    [2, 3, 4],
])  # shape (2, 3)

# Step 2: index vocab
char_matrix = vocab_arr[arr]  # shape (2, 3), dtype='<U1'

# Step 3: vectorized join — this is the key step!
joined = np.char.join('-', char_matrix)  # shape (2,), dtype='<U5' etc.

print(joined)  # Output: ['a-b-c' 'c-d-e']


In [ ]:
decoded = ['-'.join(vocab[row]) for row in arr]
decoded

In [ ]:

arr = torch.tensor([[0, 1, 2], [2, 1, 0]])
decoded = [''.join(vocab_arr[row]) for row in arr.numpy()]
print(decoded)  # ['abc', 'cba']


In [ ]:
sentences = [decode_one(seq, vocab) for seq in sequences[0,:,:].cpu().numpy()]

In [ ]:
# 1) move to CPU & to plain Python list of lists
sentences = []
for b in range(sequences.shape[0]):
    print(b)
    rows = sequences[b].cpu().tolist() # beam,T
    decoded_beams = [ctc_merge_string( ''.join(idx2char[i] for i in row) ) for row in rows]
    sentences.append(decoded_beams)
print(sentences)

In [ ]:
sentences

In [ ]:
def _decode_one( seq):
    chars = [vocab[i] for i in seq]
    raw = ''.join(chars)
    return raw


In [ ]:
sequences.shape

In [ ]:
_decode_one(sequences[0,0])

In [ ]:
"""
1.  beamctc decoder
2. 


"""